In [15]:
import skbio 
import torch
import numpy as np

In [24]:
# input 
fref_in='../../test_data/super_small_ex/ref.fasta'
freads_in='../../test_data/super_small_ex/seqs.fasta'

In [25]:
def seq_mapping(seq, alphabet):
    # Coding scheme
    # 0:A, 1:C, 2:G, 3:T 4:- (NOT YET:, 5:N)
    mapped = []
    for base in seq:
            mapped.append(alphabet.find(base))
    return np.array(mapped)

def fasta2ref(fref_in, alphabet):
    # Coding scheme
    # 0:A, 1:C, 2:G, 3:T 4:- (NOT YET:, 5:N)
    for seq in skbio.io.read(fref_in, format='fasta'):
        ref = seq_mapping(str(seq), alphabet)
    return ref

def fasta2reads(freads_in, alphabet):
    # Coding scheme
    # 0:A, 1:C, 2:G, 3:T 4:- (NOT YET:, 5:N)
    reads_mapped = []
    for seq in skbio.io.read(freads_in, format='fasta'):
        reads_mapped.append(seq_mapping(str(seq), alphabet))
    return np.array(reads_mapped)

In [23]:
alphabet ='ACGT-'
B = len(alphabet) # size alphabet

# Coding scheme
# 0:A, 1:C, 2:G, 3:T 4:-, 5:N

ref = fasta2ref(fref_in, alphabet)
reads = fasta2reads(freads_in, alphabet) # what about non-unique reads ? 

L=ref.shape[0] # length of genome
N=reads.shape[0] # number of reads

### Simplyfied model with fixed hyperparameters

ref = reference sequence  
$N$ = number of reads  
$K$ = number of clusters/components  
$L$ = genome length (number of positions)
alphabet = {A, C, G, T, -}

Fixed parameters: 
* mutation rate: $1- \gamma$
* error rate: $1-\theta$
* Dirichlet prior: $\alpha = (\alpha_1, ..., \alpha_k)$

Cluster weights ($K$-dim): $\pi | \alpha \sim Dir(\alpha)$  
Cluster assignments ($N$-dim): $z|\pi \sim Categorical(\pi)$  
Cluster centers/haplotypes ($K$x$L$-dim): $h | ref, \gamma \sim Categorical(W) $ 
with $W(l,i)=
\begin{cases} 
\gamma,  \text{ if }i = ref[l] \\
\frac{1-\gamma}{4}, \text{ else. }
\end{cases}$ for $l \in {1, ..., L}$ and $i\in {1,..., |alphabet|}$  
Likelihood of the reads ($N$-dim): $r | z, h, \theta \sim Categorical(E)$ 
with $E(n,l,i)=
\begin{cases} 
\theta,  \text{ if }i = h_{z_n}[l] \\
\frac{1-\theta}{4}, \text{ else. }
\end{cases}$ for $n \in {1, ..., N}$, $l \in {1, ..., L}$ and $i\in {1,..., |alphabet|}$  



In [27]:
import pymc3 as pm
import theano.tensor as tt
import numpy as np
import theano 

seed = 60
np.random.seed(seed)

In [44]:
# Build model 
pymc3_model = pm.Model()

# Specify fixed constants
L=ref.shape[0] # lengt}h of genome
N=reads.shape[0] # number of reads
B=len(alphabet)
K = 10  # Fixed number of components

print('K ', K)
print('N ', N)
print('B ', B)

# fixed constants
gamma = 0.95
theta = 0.98    

# --- Mutation matrix ----
# LxB-dimensional
weight = np.full((L,B),(1-gamma)/(B-1))
np.put_along_axis(weight, ref.reshape(L,1), gamma, axis=1) # is written into weight
# KxLxB dimensional 
ref_gamma_weight = np.array(K*[weight]) # KxLxB 

# --- Error matrix --- 
# NxLxB dimensional 
weight_theta = np.full((N,L,B),(1-theta)/(B-1))

theta_vec = np.full((L,B),(1-theta)/(B-1))
theta_matrix = np.asarray([np.asarray([theta_vec*L])]*N)

'''
def get_weight(theta_vec, L, B, h_z_n, theta):
    theta_matrix = theano.shared(np.asarray([theta_vec*L]) )
    print(type(h_z_n[0]))
    for l in range(L):
        theta_matrix[l][h_z_n[l]]=theta
        
    return theta_matrix

@theano.compile.ops.as_op(itypes=[tt.lvector(), tt.lmatrix],otypes=[tt.dmatrix])
def p(z, h):
    for n in range(N):
        for l in range(L):
            theta_matrix[n][l][h[z][l]]=theta
    return theta_matrix
'''
    

with pymc3_model:
     
    # prior over z (cluster sizes)
    pi = pm.Dirichlet('pi', a=np.array([1.]*K)) 
    
    # latent cluster of each of the N observations
    z = pm.Categorical('z', p=pi, shape=N)
    
    # cluster centers
    h = pm.Categorical('h', p=ref_gamma_weight, shape=[K,L])
        
    # specify the likelihood for each observation 
    weight_theta = np.full((N,L,B),(1-theta)/(B-1)) # NxLxB
    np.put_along_axis(weight_theta, h[z].reshape((N,L,1)), theta, axis=2)


    #like = pm.MixtureSameFamily('like', w=w, comp_dists=components, observed=reads, shape=N)
    #     pm.Categorical('Y_obs', p=read_weight, shape=(N,L), observed=reads)
    
    #weight = pm.Deterministic('weight', [get_weight(theta_vec, L, B, h[z][n], theta) for n in range(N)])

    Y_obs = pm.Categorical('Y_obs', p= weight_theta, shape=[N,L], observed=reads)
    #Y_obs = pm.MixtureSameFamily('Y_obs', p= h[z].reshape((N,L,1)), shape=[N,L], observed=reads)
    

K  10
N  7
B  5


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [14]:
# visulatization of the model 
pm.model_to_graphviz(pymc3_model)

## Implementation with for-loops instead of shape

In [ ]:
# Probably inefficient implementation with for-loops

# Build model 
pymc3_model = pm.Model()

# Specify fixed constants
L=ref.shape[0] # lengt}h of genome
N=reads.shape[0] # number of reads
B=len(alphabet)
K = 10  # Fixed number of components

print('K ', K)
print('N ', N)

# fixed constants
gamma = 0.95
theta = 0.98    
#alpha = torch.ones(K)
#ref_gamma_weight = get_gamma_weight(ref, gamma, B, L).expand(K,-1,-1)
ref_gamma_weight = get_gamma_weight(ref, gamma, B, L) # dimension is 
weight_theta = torch.tensor(np.full((N,L,B),(1-theta)/(B-1)))

#def p_read()

with pymc3_model:
     
    # prior over z (cluster sizes/concentrations)
    pi = pm.Dirichlet('pi', a=np.array([1.]*K))
    print(pi.distribution.shape)
    
    # cluster centers 
    h = tt.stack([pm.Categorical('h_'+str(k), p= ref_gamma_weight, shape=L) for k in range(K)])
    #centers = pm.Categorical('h', p= ref_gamma_weight, shape=[K,L])
    
    # latent cluster of each observation 
    z = tt.stack([pm.Categorical('z_' +str(n), p= pi, shape=1) for n in range(N)])
    #components = pm.Categorical('z', p= pi, shape=[N,1])
    
    print(h[0].shape.eval())
    print(z.shape)
    print(h[z].shape)
    weight = np.full((N,L,B),(1-theta)/(B-1))
    hap_temp = haplotype.detach().cpu().numpy()
    assign_temp = assignments.detach().cpu().numpy()
    hap_ass = hap_temp[assign_temp]
    np.put_along_axis(weight, hap_ass.reshape(N,L,1), theta,axis=0) 
    
    # specify the likelihood for each observation 
    like = [pm.Categorical('like'+str(n), p=p_reads(), shape=1) for n in range(N)]
    read_weight = weight_theta.scatter_(2, h[z].reshape(N,L,1), theta*torch.ones((N,L,1), dtype=torch.float64))
    like = pm.MixtureSameFamily('like', w=w, comp_dists=components, observed=reads, shape=N)
    
    pm.Categorical('Y_obs', p=read_weight, shape=(N,L), observed=reads)
    